In [17]:
import csv
import pandas as pd

In [23]:
target = 'scan/wheel_2020-09-28_20:52:50.csv'

In [24]:
# open file
f = open('../' + target, 'r')
data = list(csv.reader(f))
f.close()

In [27]:
# convert to dataframe
df = pd.DataFrame(data)
df.columns = [
    'description', 'premium', 'roc', 'req_capital', 'be', 'be_moneyness', 
    'prob_be_delta', 'prob_be_iv', 'iv', 'iv_skew', 'udl_year_ret', 
    'udl_year_ret_r2', 'udl_year_market_corr', 'udl_hist_vol', 
    'udl_iv_percentile', 'udl_hv_percentile', 'above_be_percentile'
]
df.sort_values('be_moneyness')

,description,premium,be,be_moneyness,prob_be_delta,prob_be_iv,iv,iv_skew,udl_year_ret,udl_year_ret_r2,udl_year_market_corr,udl_hist_vol,udl_iv_percentile,udl_hv_percentile,above_be_percentile
667,SOXL Nov 20 2020 $75.00 Put,30.0,74.7,0.29311,0.99329,0.97141,1.67729,0.18844,-0.021,0.25719,0.89928,0.91109,0.52727,0.60455,0.48016
409,TQQQ Nov 20 2020 $55.00 Put,90.0,54.1,0.41031,0.98196,0.94644,1.4019,0.25352,-0.01533,0.34431,0.94575,0.90309,0.75455,0.76364,0.25397
668,SOXL Nov 20 2020 $120.00 Put,225.0,117.75,0.46204,0.95405,0.92611,1.34316,0.18844,-0.021,0.25719,0.89928,0.91109,0.52727,0.60455,0.48016
410,TQQQ Nov 20 2020 $65.00 Put,165.0,63.35,0.48047,0.95493,0.92214,1.31656,0.25352,-0.01533,0.34431,0.94575,0.90309,0.75455,0.76364,0.25397
411,TQQQ Nov 20 2020 $70.00 Put,218.0,67.82,0.51437,0.94155,0.90774,1.2791,0.25352,-0.01533,0.34431,0.94575,0.90309,0.75455,0.76364,0.25397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,GLD Nov 6 2020 $175.00 Put,400.0,171.0,0.9673,0.69197,0.68056,0.21204,-0.05419,-0.05255,0.44356,0.13656,0.17367,0.60909,0.7,0.19444
362,EFA Nov 20 2020 $64.00 Put,184.0,62.16,0.96807,0.64002,0.64598,0.22217,0.33723,-0.0193,0.31339,0.92963,0.15383,0.38636,0.56364,0.67857
488,TLT Nov 20 2020 $163.00 Put,335.0,159.65,0.96922,0.68082,0.69629,0.16163,0.14513,0.00648,0.00556,-0.48433,0.10644,0.07727,0.69545,0.51587
394,HYG Nov 20 2020 $83.00 Put,183.0,81.17,0.97024,0.67652,0.71964,0.13103,0.39348,-0.01289,0.57009,0.82494,0.06118,0.44545,0.72273,0.80556


In [26]:
df

,description,premium,be,be_moneyness,prob_be_delta,prob_be_iv,iv,iv_skew,udl_year_ret,udl_year_ret_r2,udl_year_market_corr,udl_hist_vol,udl_iv_percentile,udl_hv_percentile,above_be_percentile
0,SLV Nov 6 2020 $19.00 Put,26.0,18.74,0.8472,0.86406,0.83485,0.46612,-0.11522,-0.14052,0.49568,0.30387,0.42749,0.68182,0.70909,0.19048
1,SLV Nov 6 2020 $19.50 Put,36.0,19.14,0.86528,0.8391,0.80463,0.4636,-0.11522,-0.14052,0.49568,0.30387,0.42749,0.68182,0.70909,0.19048
2,SLV Nov 6 2020 $20.00 Put,49.0,19.51,0.88201,0.81076,0.77389,0.46338,-0.11522,-0.14052,0.49568,0.30387,0.42749,0.68182,0.70909,0.19048
3,SLV Nov 6 2020 $20.50 Put,65.0,19.85,0.89738,0.78088,0.74338,0.46422,-0.11522,-0.14052,0.49568,0.30387,0.42749,0.68182,0.70909,0.19048
4,SLV Nov 6 2020 $21.00 Put,84.0,20.16,0.91139,0.75061,0.7138,0.46496,-0.11522,-0.14052,0.49568,0.30387,0.42749,0.68182,0.70909,0.19048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,SOXL Nov 20 2020 $120.00 Put,225.0,117.75,0.46204,0.95405,0.92611,1.34316,0.18844,-0.021,0.25719,0.89928,0.91109,0.52727,0.60455,0.48016
669,SOXL Nov 20 2020 $170.00 Put,890.0,161.1,0.63214,0.87862,0.83885,1.17229,0.18844,-0.021,0.25719,0.89928,0.91109,0.52727,0.60455,0.48016
670,SOXL Nov 20 2020 $210.00 Put,1920.0,190.8,0.74868,0.80027,0.75054,1.10021,0.18844,-0.021,0.25719,0.89928,0.91109,0.52727,0.60455,0.48016
671,SOXL Nov 20 2020 $225.00 Put,2480.0,200.2,0.78556,0.77007,0.71801,1.05512,0.18844,-0.021,0.25719,0.89928,0.91109,0.52727,0.60455,0.48016
